# IS 477 Midterm Notebook
### Project: Mapping and Modeling Drug-Related Deaths in Cook County

This notebook demonstrates interim progress on data acquisition, cleaning, integration, and exploratory analysis. 
It documents the steps taken so far to prepare datasets for spatial and demographic analysis, as outlined in our Project Plan.


In [5]:
# imports

import pandas as pd

In [8]:
# scripts/acquire_data.py
import pandas as pd
from sodapy import Socrata

#  client for public data
client = Socrata("datacatalog.cookcountyil.gov", None)

# Pull a sample or full dataset (limit to 50,000 for demo)
results = client.get("cjeq-bs86")
df_me = pd.DataFrame.from_records(results)

# Save as CSV
df_me.to_csv("data/raw/medical_examiner.csv", index=False)
print("Cook County Medical Examiner data acquired successfully.")


Cook County Medical Examiner data acquired successfully.


In [9]:
df_me.head()

,casenumber,incident_date,death_date,age,gender,race,latino,cold_related,heat_related,commissioner_district,...,chi_commarea,covid_related,manner,primarycause,primarycause_linea,gunrelated,opioids,primarycause_lineb,secondarycause,primarycause_linec
0,ME2025-05835,2025-11-04T14:09:00.000,2025-11-04T13:57:00.000,57,Male,Black,False,False,False,2,...,ENGLEWOOD,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ME2025-05834,2025-09-24T13:48:00.000,2025-11-04T12:20:00.000,55,Female,White,True,False,False,7,...,NEW CITY,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ME2025-05833,2025-11-04T00:00:00.000,2025-11-04T12:40:00.000,69,Female,White,True,False,False,1,...,WEST TOWN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ME2025-05832,2025-11-04T13:12:00.000,2025-11-04T12:40:00.000,75,Male,White,False,False,False,1,...,WEST TOWN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ME2025-05831,2025-11-04T13:41:00.000,2025-11-04T09:48:00.000,30,Male,Black,False,False,False,5,...,NaN,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
